In [ ]:
## conda env Weather_Prediction

import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import cartopy.crs as ccrs
import cartopy.feature as cfeature
import geopandas as gpd
import xarray as xr
import rioxarray ## what we need from rioxarray can be done using rio acessor
import regionmask

In [ ]:
## lets load the data

data = xr.open_dataset("data_sfc.nc")
data

##  Lets begin by visualizing the data

In [ ]:
## the data is multi-dimensional. the plot will be for an instant of time in a grid
# Slice the data by time and just the single variable for plotting purposes

instant = "2010-01-01T00:00:00.000000000"

instant_data = data["t2m"].sel(valid_time=slice(instant))
instant_data

In [ ]:
crs=instant_data.rio.crs ## the rio accessor
print(crs)  ## note that there no associated crs with the data

In [ ]:
## an average plot

instant_data.plot() 

In [ ]:
## improve the plot quality

fig = plt.figure(figsize=(12, 8))

p=instant_data.plot(subplot_kws=dict(projection=ccrs.Mercator()),  ## projection of the plot
                                      transform=ccrs.PlateCarree(),  ## projection of the data
                                       cbar_kwargs={"label": "2m Temperature (Kelvin)",'shrink':0.65})

p.axes.coastlines()
plt.title('2m Temperature on Jan 1 at 2010 00:00')


#### Note the fact that, in the figure below,  despite the fact that the data was downloaded over a rectangle, when it is projected in the AlbersEqualArea projection, the raster gets curved

In [ ]:
## Plot it in AlbersEqualArea projection

extent=[-125, -66.5, 48.75,24 ] ## the extent of CONUS data that is also shown in the dataset named "data"

central_lon = np.mean(extent[:2])
central_lat = np.mean(extent[2:])

fig = plt.figure(figsize=(12, 8))

p=instant_data.plot(subplot_kws=dict({'projection':ccrs.LambertAzimuthalEqualArea(central_lon, central_lat)}),
                                        transform=ccrs.PlateCarree(), 
                                          cbar_kwargs={"label": "2m Temperature (Kelvin)",'shrink':0.65} )

p.axes.coastlines()
plt.title('2m Temperature on Jan 1 2010 at 00:00')

#### Include state in the raster plot

In [ ]:
## Lets load the shapefile

SHAPE_PATH = os.path.join("State_shapefile",'cb_2018_us_state_500k.shp')
state_shape = gpd.read_file(SHAPE_PATH)
state_shape.head()


In [ ]:
print('The CRS of the SHAPE file is:', state_shape.crs)
state_shapes=state_shape.drop([13,27, 36, 37, 38,42, 44, 45 ]) ## get rid of alaska and other territories

#### Perhaps the CRS of the xarray dataset is not defined that is why combining the xarray dataset with shapefile in a different projection is not possible, for example: LambertAzimuthalEqualArea. That is the only reason I could think of. Perhaps, that is why only the default (Platecarree) projection works which has been plotted below.  

In [ ]:
fig = plt.figure(figsize=(12, 8))

# create the map using the cartopy

ax = plt.subplot(1,1,1)
instant_data.plot(cbar_kwargs={"label": "2m Temperature (Kelvin)",'shrink':0.60})
state_shapes.plot(facecolor='none', edgecolor='black', ax=ax, linewidth=0.3)
ax.set_axis_off()
ax.set_title("'2m Temperature on Jan 1 2010 at 00:00'")

